## Solve Model balancing problem with kinetic parameters as variables

In [1]:
import numpy as np
from equilibrator_api import Q_, R, default_T
import model_balancing

In [2]:
# Network topology

S = np.matrix(
    [
        [-1, 0, 0],
        [1, -1, -1],
        [0, 1, 1]
    ], dtype=float
)
A_act = np.matrix(
    [
        [0, 1, 2],
        [0, 0, 0],
        [0, 0, 0]
    ], dtype=float
)
A_inh = np.matrix(
    [
        [0, 0, 0],
        [0, 0, 0],
        [2, 0, 0]
    ], dtype=float
)

In [3]:
# Priors for Kinetic and Thermodynamic parameters

Keq_gmean = np.array([57, 0.13, 3200])  # geometric mean (assuming a standard concentration of 1M)
Keq_ln_cov = np.array(
    [
        [4, 0, 0],
        [0, 4, 0],
        [0, 0, 4],
    ]
)  # log-scale covariance

kcatf_gmean = Q_([20.0, 8.0, 0.1], "1/s")  # geometric mean
kcatf_ln_cov = np.array(
    [
        [0.48, 0, 0],
        [0, 5.3, 0],
        [0, 0, 5.3],
    ]
) # log-scale covariance

kcatr_gmean = Q_([10.0, 10.0, 10.0], "1/s")  # geometric mean
kcatr_ln_cov = np.array(
    [
        [21.2, 0, 0],
        [0, 21.2, 0],
        [0, 0, 21.2],
    ]
) # log-scale covariance

# the K parameters are only for existing connections in the S, A_act and A_inh matrices.
# the order of values is "metabolite-first".
Km_gmean = Q_([1e-2, 1e-4, 1e-4, 1e-3, 1e-1, 1e-1], "M")
Km_ln_cov = np.array(
    [
        [5.3, 0, 0, 0, 0, 0],
        [0, 5.3, 0, 0, 0, 0],
        [0, 0, 5.3, 0, 0, 0],
        [0, 0, 0, 5.3, 0, 0],
        [0, 0, 0, 0, 5.3, 0],
        [0, 0, 0, 0, 0, 5.3],
    ]
) # log-scale covariance

Ka_gmean = Q_([1, 1], "M")
Ka_ln_cov = np.array(
    [
        [21.2, 0],
        [0, 21.2],
    ]
)

Ki_gmean = Q_([1], "M")
Ki_ln_cov = np.array(
    [
        [21.2],
    ]
)

In [4]:
# condition dependent data (columns represent conditions)

fluxes = Q_(
    [
        [2.0, 1.5, 2.5, 2.5],
        [1.0, 1.0, 2.0, 0.5],
        [1.0, 0.5, 0.5, 2.0],
    ], "mM/s"
)

conc_enz_gmean = Q_(
    [
        [1e-3, 1e-3, 1e-3, 3e-3],
        [2e-3, 3e-3, 1e-3, 2e-3],
        [1e-3, 2e-3, 2e-3, 1e-3],
    ], "M")  # geometric mean

conc_enz_gsigma = np.array(
    [
        [10.0, 10.0, 10.0, 10.0],
        [10.0, 10.0, 10.0, 10.0],
        [10.0, 10.0, 10.0, 10.0],
    ]
)  # geometric standard deviation

conc_met_gmean = Q_(
    [
        [1e-4, 1e-3, 1e-4, 1e-3],
        [1e-5, 1e-5, 1e-5, 1e-4],
        [1e-4, 1e-3, 1e-4, 1e-5],
    ], "M"
)  # geometric mean

conc_met_gsigma = np.array(
    [
        [10.0, 10.0, 10.0, 10.0],
        [10.0, 10.0, 10.0, 10.0],
        [10.0, 10.0, 10.0, 10.0],
    ]
)  # geometric standard deviation

In [5]:
model_balancing.solve(
    S,
    fluxes,
    A_act,
    A_inh,
    Keq_gmean, Keq_ln_cov,
    conc_enz_gmean, conc_enz_gsigma,
    conc_met_gmean, conc_met_gsigma,
    kcatf_gmean, kcatf_ln_cov,
    kcatr_gmean, kcatr_ln_cov,
    Km_gmean, Km_ln_cov,
    Ka_gmean, Ka_ln_cov,
    Ki_gmean, Ki_ln_cov
)


Metabolite concentrations =
 [[3.46699630e-04 1.99857913e-03 4.89683215e-04 1.83832552e-03]
 [5.08116691e-05 6.86626964e-05 4.06047465e-05 1.42004248e-04]
 [6.73555838e-05 1.59069310e-04 5.34650337e-05 9.99953632e-06]]

Enzyme concentrations =
 [[0.00052205 0.00012086 0.00047957 0.00023156]
 [0.01313218 0.01037386 0.02231131 0.00042771]
 [0.00540642 0.0007897  0.00233501 0.00218516]]

eta_thr =
 [[0.99 1.   1.   1.  ]
 [0.49 0.11 0.5  0.97]
 [1.   1.   1.   1.  ]]

eta_kin =
 [[0.16 0.5  0.21 0.44]
 [0.29 0.35 0.24 0.53]
 [0.31 0.37 0.26 0.55]]

eta_reg =
 [[1.   1.   1.   1.  ]
 [0.06 0.27 0.08 0.25]
 [0.26 0.74 0.36 0.72]]




All Z-scores
--------------------------------------------------
enzymes =  3.43
metabolites =  3.26
ΔG'0 =  2.43
kcat forward =  1.95
kcat reverse =  1.89
Km =  2.04
Ka =  4.32
Ki =  0.0


In [6]:
model_balancing.solve(
    S,
    fluxes,
    A_act,
    A_inh,
    Keq_gmean, Keq_ln_cov,
    conc_enz_gmean, conc_enz_gsigma,
    conc_met_gmean, conc_met_gsigma,
    kcatf_gmean, kcatf_ln_cov,
    kcatr_gmean, kcatr_ln_cov,
    Km_gmean, Km_ln_cov,
    Ka_gmean, Ka_ln_cov,
    Ki_gmean, Ki_ln_cov,
    rate_law = "1SP"
)


Metabolite concentrations =
 [[3.82310493e-04 1.44030577e-03 5.77065604e-04 1.49439693e-03]
 [4.39118557e-05 7.84100304e-05 4.72655142e-05 1.50285989e-04]
 [6.30997848e-05 1.76886973e-04 5.49302835e-05 9.99945188e-06]]

Enzyme concentrations =
 [[0.00055745 0.00016419 0.00049865 0.00029889]
 [0.01310194 0.0111281  0.01364479 0.00043879]
 [0.00679382 0.0009446  0.00223394 0.00270082]]

eta_thr =
 [[1.   1.   1.   1.  ]
 [0.43 0.11 0.54 0.97]
 [1.   1.   1.   1.  ]]

eta_kin =
 [[0.14 0.37 0.2  0.34]
 [0.28 0.41 0.29 0.57]
 [0.3  0.43 0.31 0.59]]

eta_reg =
 [[1.   1.   1.   1.  ]
 [0.07 0.22 0.1  0.23]
 [0.25 0.63 0.36 0.64]]




All Z-scores
--------------------------------------------------
enzymes =  3.16
metabolites =  3.35
ΔG'0 =  2.4
kcat forward =  1.78
kcat reverse =  2.01
Km =  1.99
Ka =  4.24
Ki =  0.0
